In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np
from funcs import constants
hv.notebook_extension()

def add_energy_gs(df):
    flux_quantum_over_2pi = constants.hbar / (2 * constants.eV) / (constants.eV * 1e6)
    df['E'] = df['currents'].apply(np.cumsum)
    df['E'] *= flux_quantum_over_2pi
    df['phase_gs_arg'] = df['E'].apply(np.argmin)
    df['phase_gs'] = [row['phases'][row['phase_gs_arg']] for i, row in df.iterrows()]

    # Move the phase_gs from -π to +π if they are within the tolerance
    tol = np.diff(df['phases'].iloc[0]).max()
    df['phase_gs'] = [-row['phase_gs'] if row['phase_gs'] < -(np.pi - tol) else row['phase_gs'] 
                      for i, row in df.iterrows()]
    return df

df = pd.read_hdf('data/I_c(B_x)_mu10,20meV_disorder0,75meV_T0.1K_c5over8_only_combinations_in_paper.hdf')
df = add_energy_gs(df)
params = ['orbital', 'g', 'alpha', 'mu', 'disorder', 'salt']
gb = df.groupby(params)

In [ ]:
%%opts Curve (color='k') Scatter (s=200)

def plot(orbital, g, alpha, mu, disorder, salt, B_x):
    gr = gb.get_group((orbital, g, alpha, mu, disorder, salt))
    gr = gr.set_index('B_x', drop=False)
    x = gr.loc[B_x]
    current = hv.Curve((gr.B_x, gr.current_c), kdims=['B_x'], vdims=['I_c'])
    phase_gs = hv.Curve((gr.B_x, gr.phase_gs), kdims=['B_x'], vdims=['theta_gs'])
    cpr = hv.Curve((x.phases, x.currents), kdims=['phi'], vdims=['I'])
    energy = hv.Curve((x.phases, x.E), kdims=['phi'], vdims=['I'])
    E_min = hv.Scatter((x.phase_gs, x.E[x.phase_gs_arg]), kdims=['phi'], vdims=['I'])
    VLine = hv.VLine(B_x)
    return (current * VLine + phase_gs * VLine + cpr + energy * E_min).cols(2)

kdims = [hv.Dimension('orbital', values=df.orbital.unique()), 
         hv.Dimension('g', values=df.g.unique()), 
         hv.Dimension('alpha', values=df.alpha.unique()), 
         hv.Dimension('mu', values=df.mu.unique()), 
         hv.Dimension('disorder', values=df.disorder.unique()), 
         hv.Dimension('salt', values=df.salt.unique()), 
         hv.Dimension('B_x', values=df.B_x.unique())]

hv.DynamicMap(plot, kdims=kdims)